In [39]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score, mean_absolute_percentage_error, r2_score
from bayes_opt import BayesianOptimization
from sklearn.model_selection import cross_val_score
from lightgbm import plot_importance
import matplotlib.pyplot as plt
import torch
from torch import nn
import copy
import joblib
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler

In [41]:
class Soma_old(nn.Module):
    def __init__(self, kt, qs):
        super(Soma_old, self).__init__()
        self.kt = kt
        self.qs = qs

    def forward(self, x):
        y = 1 / (1 + torch.exp(-self.kt * (x - self.qs)))
#         y = x
        return y

class Membrane(nn.Module):
    def __init__(self,b):
        super(Membrane, self).__init__()
        self.params = nn.ParameterDict({'b': nn.Parameter(b)})

    def forward(self, x):
        x = torch.mul(x,self.params['b'])
        x = torch.sum(x, 1)
        return x

class Dendritic(nn.Module):
    def __init__(self):
        super(Dendritic, self).__init__()

    def forward(self, x):
        x = torch.prod(x, 2)
        return x


class Synapse_old(nn.Module):

    def __init__(self, w, q, k):
        super(Synapse_old, self).__init__()
        self.params = nn.ParameterDict({'w': nn.Parameter(w)})
        self.params.update({'q': nn.Parameter(q)})
        self.k = k

    def forward(self, x):
        num, _ = self.params['w'].shape
        x = torch.unsqueeze(x, 1)
        x = x.repeat((1, num, 1))
        y = 1 / (1 + torch.exp(
            torch.mul(-self.k, (torch.mul(x, self.params['w']) - self.params['q']))))
        return y

class DNM_Model(nn.Module):
    def __init__(self, w, q, k, kt, b,qs):
        super(DNM_Model, self).__init__()
        self.model = nn.Sequential(
            Synapse_old(w, q, k),
            Dendritic(),
            Membrane(b),
            Soma_old(k, qs)
        )
        self.w = nn.Parameter(w)
        self.q = nn.Parameter(q)
        self.k = k
        self.kt = kt
        self.b = nn.Parameter(b)
        self.qs = qs

    def forward(self, x):
        x = self.model(x)
        return x

In [42]:
def train(X_train, y_train, batch_size, model, loss_func, optimizer, DEVICE):
    model.train()
    perm = torch.randperm(X_train.shape[0])
    sum_loss = 0
    count = 0
    for i in range(0, X_train.shape[0], batch_size):
        indices = perm[i:i + batch_size]
        batch_X, batch_y = X_train[indices].to(DEVICE), y_train[indices].to(DEVICE)
        y_pred = model(batch_X)
        loss = loss_func(y_pred, batch_y)
        sum_loss += loss.item()
        count += 1
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    average_loss = sum_loss / count
    return average_loss

def test(X_test, y_test, model, loss_func, DEVICE):
    model.eval()
    with torch.no_grad():
        y_pred = model(X_test)
        loss = loss_func(y_pred, y_test)
    return loss.item()

In [43]:
def load_and_preprocess_data(filepath):
    df = pd.read_csv(filepath)
    z_scores = (df - df.mean()) / df.std()
    outliers = (z_scores.abs() > 3).any(axis=1)
    df = df[~outliers]
    data = df.to_numpy()
    X = data[:, :-1]
    Y = data[:, -1]

    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    X = torch.tensor(X, dtype=torch.float32).to(DEVICE)
    Y = torch.tensor(Y, dtype=torch.float32).to(DEVICE)

    x_max, _ = torch.max(X, dim=0)
    x_min, _ = torch.min(X, dim=0)
    X = (X - x_min) / (x_max - x_min)

    y_max, _ = torch.max(Y, dim=0)
    y_min, _ = torch.min(Y, dim=0)
    Y = (Y - y_min) / (y_max - y_min)

    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=23)
    return X_train, X_test, y_train, y_test

In [40]:
def MAE_loss(y_pred, y_true):
    error = torch.abs(y_pred - y_true)
    return torch.mean(error)


def MSE_loss(y_pred, y_true):
    error = torch.square(y_pred - y_true)
    error = torch.mean(error)
    return error


def R_square_loss(y_pred, y_true):
    y_mean = torch.mean(y_true)
    ss_res = torch.sum(torch.pow(y_true - y_pred, 2))
    ss_tot = torch.sum(torch.pow(y_true - y_mean, 2))
    r2 = 1 - ss_res / ss_tot
    return r2

In [46]:
def main():
    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    epochs = 800
    batch_size = 64
    X_train, X_test, y_train, y_test = load_and_preprocess_data('data_g.csv')

    M, D = 33, X_train.shape[1]
    torch.cuda.manual_seed(20)
    torch.cuda.manual_seed_all(20) 
    w = torch.randn(M, D, dtype=torch.float32, device=DEVICE)
    q = torch.randn(M, D, dtype=torch.float32, device=DEVICE)
    b = torch.randn(M, dtype=torch.float32, device=DEVICE)
    k = 1
    kt=torch.randn(1, dtype=torch.float32, device=DEVICE)
    qs=torch.randn(1, dtype=torch.float32, device=DEVICE)
    model = DNM_Model(w, q, k,kt, b, qs).to(DEVICE)
    optimizer = torch.optim.Adamax(model.parameters(), lr=0.01)
    criterion = torch.nn.MSELoss()

    best_loss = float('inf')
    best_model = None

    for epoch in range(epochs):
        train(X_train, y_train, batch_size, model, criterion, optimizer, DEVICE)
        loss = test(X_test, y_test, model, criterion, DEVICE)
        if loss < best_loss:
            best_loss = loss
            best_model = copy.deepcopy(model)
            joblib.dump(best_model, "DNM_model.dat") 
        if epoch % 100 == 0:
            print(f'Epoch [{epoch+1}/{epochs}], Loss on Test Data: {best_loss:.8f}')
    
    model = joblib.load("DNM_model.dat")
    model.eval()  
    with torch.no_grad():  
        y_pred = model(X_test)
    y_pred = torch.tensor(y_pred)
    print(R_square_loss(y_pred, y_test))
if __name__ == "__main__":
    main()

Epoch [1/800], Loss on Test Data: 0.02590176
Epoch [101/800], Loss on Test Data: 0.00661080
Epoch [201/800], Loss on Test Data: 0.00657466
Epoch [301/800], Loss on Test Data: 0.00657076
Epoch [401/800], Loss on Test Data: 0.00657076
Epoch [501/800], Loss on Test Data: 0.00657076


KeyboardInterrupt: 

In [53]:
X_train, X_test, y_train, y_test = load_and_preprocess_data('data_g.csv')
model = joblib.load("DNM_model.dat")
model.eval()  
with torch.no_grad():  
    y_pred = model(X_test)
y_pred = torch.tensor(y_pred)
R_square_loss(y_pred, y_test)

C:\Users\13054\AppData\Local\Temp\ipykernel_28348\2337591828.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_pred = torch.tensor(y_pred)


tensor(0.7582, device='cuda:0')

In [63]:
# sensitivity analysis
def sensitivity_analysis_with_perturbation(X_test, perturbation_factor=0.05):

    # 在每个特征上进行扰动
    perturbation = perturbation_factor * torch.randn_like(X_test)
    X_test_perturbed = X_test + perturbation
    return X_test_perturbed

def main():
    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    X_train, X_test, y_train, y_test = load_and_preprocess_data('data_g.csv')

    # 加载已经训练好的模型
    model = joblib.load("DNM_model.dat")
    model.eval()  # 设置为评估模式
    
    # 进行 10 次扰动分析和 R² 输出
    for run in range(10):
        print(f"\nRun {run + 1} / 10:")

        # 执行扰动分析
        X_test_perturbed = sensitivity_analysis_with_perturbation(X_test, perturbation_factor=0.05)

        # 使用模型进行预测
        with torch.no_grad():  # 确保不会计算梯度
            y_pred = model(X_test_perturbed)
        
        # 计算 R²
        y_pred = y_pred.cpu().numpy()
        y_test_1 = y_test.cpu().numpy()
        r2 = r2_score(y_test_1, y_pred)
        
        print(f"R² Score: {r2:.6f}")

if __name__ == "__main__":
    main()



Run 1 / 10:
R² Score: 0.710243

Run 2 / 10:
R² Score: 0.711555

Run 3 / 10:
R² Score: 0.708212

Run 4 / 10:
R² Score: 0.706669

Run 5 / 10:
R² Score: 0.712976

Run 6 / 10:
R² Score: 0.711760

Run 7 / 10:
R² Score: 0.707085

Run 8 / 10:
R² Score: 0.704024

Run 9 / 10:
R² Score: 0.714118

Run 10 / 10:
R² Score: 0.712069


In [66]:
# Comparison of optimization algorithms
# SGD
def main():
    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    epochs = 300
    batch_size = 64
    X_train, X_test, y_train, y_test = load_and_preprocess_data('data_g.csv')

    M, D = 33, X_train.shape[1]
    torch.cuda.manual_seed(20)
    torch.cuda.manual_seed_all(20) 
    w = torch.randn(M, D, dtype=torch.float32, device=DEVICE)
    q = torch.randn(M, D, dtype=torch.float32, device=DEVICE)
    b = torch.randn(M, dtype=torch.float32, device=DEVICE)
    k = 1
    kt=torch.randn(1, dtype=torch.float32, device=DEVICE)
    qs=torch.randn(1, dtype=torch.float32, device=DEVICE)
    model = DNM_Model(w, q, k,kt, b, qs).to(DEVICE)
    optimizer = torch.optim.SGD(model.parameters(), lr=0.1)
    criterion = torch.nn.MSELoss()

    best_loss = float('inf')
    best_model = None

    for epoch in range(epochs):
        train(X_train, y_train, batch_size, model, criterion, optimizer, DEVICE)
        loss = test(X_test, y_test, model, criterion, DEVICE)
        if loss < best_loss:
            best_loss = loss
            best_model = copy.deepcopy(model)
            joblib.dump(best_model, "DNM_model.dat") 
        if epoch % 100 == 0:
            print(f'Epoch [{epoch+1}/{epochs}], Loss on Test Data: {best_loss:.8f}')
    
    model = joblib.load("DNM_model.dat")
    model.eval()  
    with torch.no_grad():  
        y_pred = model(X_test)
    y_pred = torch.tensor(y_pred)
    print(R_square_loss(y_pred, y_test))
if __name__ == "__main__":
    main()

Epoch [1/300], Loss on Test Data: 0.05200727
Epoch [101/300], Loss on Test Data: 0.05112347
Epoch [201/300], Loss on Test Data: 0.02193140
tensor(0.5182, device='cuda:0')


C:\Users\13054\AppData\Local\Temp\ipykernel_28348\2118694101.py:38: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_pred = torch.tensor(y_pred)


In [67]:
# Adagrad
def main():
    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    epochs = 300
    batch_size = 64
    X_train, X_test, y_train, y_test = load_and_preprocess_data('data_g.csv')

    M, D = 33, X_train.shape[1]
    torch.cuda.manual_seed(20)
    torch.cuda.manual_seed_all(20) 
    w = torch.randn(M, D, dtype=torch.float32, device=DEVICE)
    q = torch.randn(M, D, dtype=torch.float32, device=DEVICE)
    b = torch.randn(M, dtype=torch.float32, device=DEVICE)
    k = 1
    kt=torch.randn(1, dtype=torch.float32, device=DEVICE)
    qs=torch.randn(1, dtype=torch.float32, device=DEVICE)
    model = DNM_Model(w, q, k,kt, b, qs).to(DEVICE)
    optimizer = torch.optim.Adagrad(model.parameters(), lr=0.05)
    criterion = torch.nn.MSELoss()

    best_loss = float('inf')
    best_model = None

    for epoch in range(epochs):
        train(X_train, y_train, batch_size, model, criterion, optimizer, DEVICE)
        loss = test(X_test, y_test, model, criterion, DEVICE)
        if loss < best_loss:
            best_loss = loss
            best_model = copy.deepcopy(model)
            joblib.dump(best_model, "DNM_model.dat") 
        if epoch % 100 == 0:
            print(f'Epoch [{epoch+1}/{epochs}], Loss on Test Data: {best_loss:.8f}')
    
    model = joblib.load("DNM_model.dat")
    model.eval()  
    with torch.no_grad():  
        y_pred = model(X_test)
    y_pred = torch.tensor(y_pred)
    print(R_square_loss(y_pred, y_test))
if __name__ == "__main__":
    main()

Epoch [1/300], Loss on Test Data: 0.01551471
Epoch [101/300], Loss on Test Data: 0.00675118
Epoch [201/300], Loss on Test Data: 0.00671349
tensor(0.7538, device='cuda:0')


C:\Users\13054\AppData\Local\Temp\ipykernel_28348\2756982503.py:38: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_pred = torch.tensor(y_pred)


In [68]:
# Adam
def main():
    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    epochs = 300
    batch_size = 64
    X_train, X_test, y_train, y_test = load_and_preprocess_data('data_g.csv')

    M, D = 33, X_train.shape[1]
    torch.cuda.manual_seed(20)
    torch.cuda.manual_seed_all(20) 
    w = torch.randn(M, D, dtype=torch.float32, device=DEVICE)
    q = torch.randn(M, D, dtype=torch.float32, device=DEVICE)
    b = torch.randn(M, dtype=torch.float32, device=DEVICE)
    k = 1
    kt=torch.randn(1, dtype=torch.float32, device=DEVICE)
    qs=torch.randn(1, dtype=torch.float32, device=DEVICE)
    model = DNM_Model(w, q, k,kt, b, qs).to(DEVICE)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    criterion = torch.nn.MSELoss()

    best_loss = float('inf')
    best_model = None

    for epoch in range(epochs):
        train(X_train, y_train, batch_size, model, criterion, optimizer, DEVICE)
        loss = test(X_test, y_test, model, criterion, DEVICE)
        if loss < best_loss:
            best_loss = loss
            best_model = copy.deepcopy(model)
            joblib.dump(best_model, "DNM_model.dat") 
        if epoch % 100 == 0:
            print(f'Epoch [{epoch+1}/{epochs}], Loss on Test Data: {best_loss:.8f}')
    
    model = joblib.load("DNM_model.dat")
    model.eval()  
    with torch.no_grad():  
        y_pred = model(X_test)
    y_pred = torch.tensor(y_pred)
    print(R_square_loss(y_pred, y_test))
if __name__ == "__main__":
    main()

Epoch [1/300], Loss on Test Data: 0.01595180
Epoch [101/300], Loss on Test Data: 0.00658613
Epoch [201/300], Loss on Test Data: 0.00658613
tensor(0.7586, device='cuda:0')


C:\Users\13054\AppData\Local\Temp\ipykernel_28348\3161610075.py:38: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_pred = torch.tensor(y_pred)


In [69]:
# Adam
def main():
    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    epochs = 300
    batch_size = 64
    X_train, X_test, y_train, y_test = load_and_preprocess_data('data_g.csv')

    M, D = 33, X_train.shape[1]
    torch.cuda.manual_seed(20)
    torch.cuda.manual_seed_all(20) 
    w = torch.randn(M, D, dtype=torch.float32, device=DEVICE)
    q = torch.randn(M, D, dtype=torch.float32, device=DEVICE)
    b = torch.randn(M, dtype=torch.float32, device=DEVICE)
    k = 1
    kt=torch.randn(1, dtype=torch.float32, device=DEVICE)
    qs=torch.randn(1, dtype=torch.float32, device=DEVICE)
    model = DNM_Model(w, q, k,kt, b, qs).to(DEVICE)
    optimizer = torch.optim.Adamax(model.parameters(), lr=0.01)
    criterion = torch.nn.MSELoss()

    best_loss = float('inf')
    best_model = None

    for epoch in range(epochs):
        train(X_train, y_train, batch_size, model, criterion, optimizer, DEVICE)
        loss = test(X_test, y_test, model, criterion, DEVICE)
        if loss < best_loss:
            best_loss = loss
            best_model = copy.deepcopy(model)
            joblib.dump(best_model, "DNM_model.dat") 
        if epoch % 100 == 0:
            print(f'Epoch [{epoch+1}/{epochs}], Loss on Test Data: {best_loss:.8f}')
    
    model = joblib.load("DNM_model.dat")
    model.eval()  
    with torch.no_grad():  
        y_pred = model(X_test)
    y_pred = torch.tensor(y_pred)
    print(R_square_loss(y_pred, y_test))
if __name__ == "__main__":
    main()

Epoch [1/300], Loss on Test Data: 0.02518431
Epoch [101/300], Loss on Test Data: 0.00661515
Epoch [201/300], Loss on Test Data: 0.00658264
tensor(0.7581, device='cuda:0')


C:\Users\13054\AppData\Local\Temp\ipykernel_28348\1126140402.py:38: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_pred = torch.tensor(y_pred)
